In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import torch
import timm
import pandas as pd
import os
from torch.utils.data import Dataset,DataLoader
from PIL import Image
from torchvision.transforms import transforms
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

In [3]:
class loader(Dataset):
    def __init__(self,ROOT,transform):
        self.data = annotation = pd.read_csv(os.path.join(ROOT,'test.csv'))["path"].values
        self.ROOT = ROOT
        self.transform = transform
    def __len__(self):
        return len(self.data)
    def __getitem__(self,idx):
        id = self.data[idx].split('/')[-1]
        path = os.path.join(ROOT,'test/test_images',id)
        img = transform(Image.open(path).convert('RGB'))
        return img

In [4]:
ROOT = '/kaggle/input/bird-classification-anokha'

In [5]:
data = loader(ROOT, transform)
test_data = DataLoader(data, batch_size=64)

PRETRAINED = '/kaggle/input/model-weight/1student_model45.pth'
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load the EdgeNeXt model
model = timm.create_model('edgenext_xx_small', pretrained=False)
model.reset_classifier(num_classes=200)
model.to(device)

# Load the model's state dict correctly
checkpoint = torch.load(PRETRAINED, map_location=device,weights_only=True)

# If the checkpoint has a 'model_state_dict', use that key
if 'model_state_dict' in checkpoint:
    model.load_state_dict(checkpoint['model_state_dict'])
else:
    model.load_state_dict(checkpoint)

print("Model loaded successfully.")


Model loaded successfully.


In [6]:
def test_model(model, test_loader):
   
    predictions = []

    with torch.no_grad():
        for images in test_loader:
            images = images.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            predictions.extend(predicted.cpu().numpy())

    return predictions


predictions = test_model(model, test_data)

test_df = pd.read_csv('/kaggle/input/bird-classification-anokha/test.csv')
test_df=test_df.drop('bbox',axis=1)



test_df['class'] = predictions
test_df.to_csv('submission.csv', index=False)
